Data Retrieval

Installing required libraries

In [0]:
%pip install langchain faiss-cpu databricks-langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/30.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 30.7/30.7 MB 209.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/28.4 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 155.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/6.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

Importing required libraries

In [0]:
from databricks_langchain import ChatDatabricks
from langchain_community.vectorstores import FAISS
from databricks_langchain import DatabricksEmbeddings

import os
import faiss

Setting up initial configuration

In [0]:
if 'config' not in locals(): config = {}

config['VECTOR_STORE_PATH'] = f"/Volumes/practice/default/rag_t2_vector_store"

embeddings = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

llm = ChatDatabricks(
    endpoint="databricks-dbrx-instruct",
    temperature=0.1,
)

Function Definitions

Wrapping Functions

In [0]:
def load_vector_store():
    """
    Load the vector_store stored locally at config['VECTOR_STORE_PATH']
    """
    try:
        vector_store = FAISS.load_local(
            folder_path=config['VECTOR_STORE_PATH'],
            embeddings=embeddings,
            allow_dangerous_deserialization=True  # Fix for pickle loading
        )

        return vector_store
    except Exception as e:
        print(f"Error loading vector_store: {e}")
       
        return None

In [0]:
def retrieve_data(query, vector_store):
    """
    Searches the vector store for data similar to the query.

    Args:
        query (str): Input query.

    Returns:
        List of similar data retrieved from the vector store.
    """
    try:
        retrieved_data = vector_store.similarity_search(query)

        return retrieved_data
    except Exception as e:
        print(f"Error retrieving data: {e}")
       
        return []

In [0]:
def generate_answer(query, retrieved_data):
    """
    Generates an answer using an LLM based on the user query and retrieved documents.

    Args:
        query (str): The user's question.
        retrieved_data (str): The most relevant retrieved information related to the query.

    Returns:
        str: The AI-generated response.
    """
    messages = [
            (
                "system",
                """You are an AI assistant that provides helpful responses using retrieved documents and user queries.
                Try to give answers that are true and verified. If you don't know the answer, say so. Keep the asnwer
                concise and to the point""",
            ),
            (
                "human",
                f"User Query: {query}\n\nRetrieved Documents:\n{retrieved_data}",
            ),
        ]

    try:
        response = llm.invoke(messages)

        return response
    except Exception as e:
        return f"An error occurred while generating a response: {e}"

Main

In [0]:
print("\nStep: Loading vector store")
vector_store = load_vector_store()


Step: Loading vector store


In [0]:
# print("Step: Get user query")
# query = input("Enter your query:")
query = "What is the current gold price ?"

In [0]:
print("\nStep: Retrieving similar data")
retrieved_data = retrieve_data(query, vector_store)
print("Retrieved Data:", retrieved_data)


Step: Retrieving similar data
Retrieved Data: [Document(id='8a3a3535-fe45-4844-ae00-debc4b24a288', metadata={'source': '/Volumes/practice/default/datasets/pdf/electric_vehicles.pdf', 'start_index': 5535}, page_content='especially if you can charge your EV at home.\n\nUp to 40% lower servicing and maintenance costs than petrol or diesel vehicles due\n\nto fewer mechanical components.\n\nLower vehicle exercise duty (VED) rates until 2030, for new electric cars registered\n\non or after 1 April 2025.\n\no Currently it costs £10 for the first year, compared with £120-£945 for petrol or diesel vehicles. After that, all vehicles are charged a standard rate of £165 per year.\n\no For the 2025 to 2026 tax year, electric cars will continue paying £10 for the\n\nfirst year, compared with £110-£1,000+ for hybrid, petrol and diesel vehicles. The more tailpipe emissions produced, the more it will cost. After that, all vehicles will be charged a standard rate of £195 per year.\n\nAs they emit zero 

In [0]:
print("\nStep: Generating a response using LLM")
response = generate_answer(query, retrieved_data) ## loads similar data from vector store
print("Response:", response.content)


Step: Generating a response using LLM
Response: Based on the retrieved documents, the current gold price is not mentioned. The documents discuss the benefits and costs of electric vehicles. For example, a full charge in a fully electric car costs approximately £17 and gives a typical range of around 220 miles. However, for the most accurate and up-to-date information on the gold price, I would recommend checking a reliable financial news source.
